In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the uploaded CSV file into a DataFrame
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# Display the first few rows
data.head()



Saving Soil_merged.csv to Soil_merged (3).csv


,fid,Survey_No,STATE,DISTRICT,TEHSIL,pH,EC,OC,n,p,...,field_id,polarization_diff_s1,ndvi_s2,bsi_s2,osavi_s2,gndvi_s2,sr1_s2,sr2_s2,ndwi_s2,mean_nir_band_s2
0,1,11-Jan,Gujrat,NaN,NaN,7.91,0.22,0.42,NaN,10.0,...,284,17.300871,0.142846,0.205670,0.090762,0.322064,1.334322,0.720174,-0.322064,0.162560
1,2,11-Feb,Gujrat,NaN,NaN,8.09,0.19,0.60,NaN,6.0,...,283,21.555414,0.145391,0.192945,0.093909,0.346883,1.347734,0.675383,-0.346883,0.168674
2,3,140,Gujrat,NaN,NaN,8.22,0.30,0.54,NaN,6.0,...,310,22.132469,0.127440,0.202547,0.085367,0.328108,1.294065,0.671539,-0.328108,0.189712
3,4,22,Gujrat,NaN,NaN,7.46,0.56,0.54,NaN,10.0,...,272,18.114547,0.151544,0.192670,0.098100,0.327093,1.360491,0.743541,-0.327093,0.173924
4,5,48,Gujrat,NaN,NaN,7.74,0.31,0.52,NaN,14.0,...,301,10.778053,0.149600,0.188388,0.103821,0.299159,1.353599,0.747119,-0.299159,0.217503


In [ ]:
# Select relevant satellite feature columns
satellite_features = [
    'polarization_diff_s1', 'ndvi_s2', 'bsi_s2', 'osavi_s2',
    'gndvi_s2', 'sr1_s2', 'sr2_s2', 'ndwi_s2', 'mean_nir_band_s2'
]

# Create a new column with string representations of the satellite data
data['input_string'] = data[satellite_features].apply(lambda row: ', '.join(
    [f"{col}: {row[col]:.4f}" for col in satellite_features]), axis=1)

# Append the target variable 'P' to each string
data['input_string'] = data['input_string'] + ' => P: ' + data['p'].astype(str)

# Display a few examples of the input strings
data[['input_string']].head()


,input_string
0,"polarization_diff_s1: 17.3009, ndvi_s2: 0.1428..."
1,"polarization_diff_s1: 21.5554, ndvi_s2: 0.1454..."
2,"polarization_diff_s1: 22.1325, ndvi_s2: 0.1274..."
3,"polarization_diff_s1: 18.1145, ndvi_s2: 0.1515..."
4,"polarization_diff_s1: 10.7781, ndvi_s2: 0.1496..."


In [ ]:
from datasets import Dataset

# Convert the DataFrame into a Hugging Face dataset
dataset = Dataset.from_pandas(data[['input_string']])

# Display the dataset structure
dataset


Dataset({
    features: ['input_string'],
    num_rows: 480
})

In [ ]:
from transformers import GPT2Tokenizer
import torch

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Padding token

# Define tokenization function
def tokenize_function(examples):
    tokenized = tokenizer(examples['input_string'], padding="max_length", truncation=True)
    tokenized['labels'] = tokenized['input_ids'].copy()  # Labels for GPT-2 are just the input shifted
    return tokenized

# Tokenize the dataset and map input_ids to labels
tokenized_data = dataset.map(tokenize_function, batched=True)

# Set the format to return tensors, as required for PyTorch models
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [ ]:
# Import necessary libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Tokenization setup using distilgpt2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Add padding token if missing

# Load your dataset (ensure you've already preprocessed and tokenized it)
# Assuming tokenized_data exists

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Changed 'evaluation_strategy' to 'eval_strategy' to avoid deprecation warning
    per_device_train_batch_size=2,  # Adjust batch size to prevent memory issues
    gradient_accumulation_steps=8,  # Simulate a larger batch size by accumulating gradients
    num_train_epochs=3,
    weight_decay=0.01
)

# Load the distilgpt2 model (lighter version of GPT-2)
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,  # You may want to split the data for evaluation
)

# Start training
trainer.train()


Epoch,Training Loss,Validation Loss
